In [21]:
# Import libraries for data manipulation and ETL pipeline
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from pandarallel import pandarallel
pandarallel.initialize(verbose=0)
from tqdm import tqdm

In [22]:
# Establish API connection
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid = '3fda75b7146a4769b207ee44017b3abe'
secret = '2a755cb04a18406b9394dbef2f8069dd'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=15)

In [23]:
# Connect to SQL Server
import sqlalchemy as sql
from sqlalchemy import text
import pyodbc
server = 'LAPTOP-V3754MEK' 
database = 'Spotify'

engine = sql.create_engine('mssql+pyodbc://' + server + '/' + database + '?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server')
conn = engine.connect()
results = engine.execute("SELECT track_uri2 FROM dim_tracks ORDER BY track_uri2")
results = pd.Series(results)
results.head()

0    (spotify:track:0002yNGLtYSYtc0X6ZnFvp)
1    (spotify:track:00039MgrmLoIzSpuYKurn9)
2    (spotify:track:0003Z98F6hUq7XxqSRM87H)
3    (spotify:track:0004ExljAge0P5XWn1LXmW)
4    (spotify:track:0005rgjsSeVLp1cze57jIN)
dtype: object

In [24]:
# Extract tracks from API
output_df = pd.DataFrame()
for row in tqdm(results[1510001:1530001]):
      output = pd.DataFrame(sp.audio_features(row))
      output_df = pd.concat([output_df, output])
output_df.head()

100%|██████████| 20000/20000 [36:35<00:00,  9.11it/s] 


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,0
0,0.737,0.837,7.0,-4.802,1.0,0.0394,0.02890,0.840000,0.2490,0.637,91.014,audio_features,5b0clm7fxE8FxuTqT0AS8t,spotify:track:5b0clm7fxE8FxuTqT0AS8t,https://api.spotify.com/v1/tracks/5b0clm7fxE8F...,https://api.spotify.com/v1/audio-analysis/5b0c...,210600.0,4.0,NaN
0,0.530,0.419,11.0,-8.381,1.0,0.0297,0.13600,0.000041,0.1190,0.201,134.620,audio_features,5b0cMj6EiDZrl4LrgpFaXB,spotify:track:5b0cMj6EiDZrl4LrgpFaXB,https://api.spotify.com/v1/tracks/5b0cMj6EiDZr...,https://api.spotify.com/v1/audio-analysis/5b0c...,232750.0,4.0,NaN
0,0.583,0.912,2.0,-5.578,1.0,0.1400,0.18700,0.000000,0.2430,0.522,79.998,audio_features,5B0crlj9F66CwVDQuHSOSv,spotify:track:5B0crlj9F66CwVDQuHSOSv,https://api.spotify.com/v1/tracks/5B0crlj9F66C...,https://api.spotify.com/v1/audio-analysis/5B0c...,192360.0,4.0,NaN
0,0.696,0.839,5.0,-9.251,1.0,0.0300,0.01760,0.000036,0.0551,0.821,120.723,audio_features,5B0CwHhUa4lSOzcgDINenE,spotify:track:5B0CwHhUa4lSOzcgDINenE,https://api.spotify.com/v1/tracks/5B0CwHhUa4lS...,https://api.spotify.com/v1/audio-analysis/5B0C...,194000.0,4.0,NaN
0,0.732,0.654,6.0,-9.287,0.0,0.1110,0.00483,0.645000,0.0680,0.167,70.044,audio_features,5B0cxJ6eGpeLY55lcHAR1M,spotify:track:5B0cxJ6eGpeLY55lcHAR1M,https://api.spotify.com/v1/tracks/5B0cxJ6eGpeL...,https://api.spotify.com/v1/audio-analysis/5B0c...,391841.0,4.0,NaN


In [25]:
# Load data into SQL Server
output_df = output_df[['uri', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
output_df = output_df.rename(columns={'key':'music_key', 'mode':'modal'})
output_df.to_sql("audio_features2", engine, if_exists='append', index=False)
print(f'{len(output_df)} rows imported to database.')
new_query = engine.execute('SELECT COUNT(DISTINCT uri) FROM audio_features2')
new_query = pd.Series(new_query)
print(f'App contains {new_query[0]} songs.')

20000 rows imported to database.
App contains (1519929,) songs.
